# LoRa, TTN and CircuitPython

### Connect to CircuitPythonboard

In [192]:
%serialconnect to /dev/tty.usbmodem14601 --baud=115200

Found serial ports: /dev/cu.usbmodem14601, /dev/cu.AirPodsdeAlvaro-Wireles-2, /dev/cu.BTSPEAKER-COM3-1, /dev/cu.Bluetooth-Incoming-Port, /dev/cu.MALS, /dev/cu.SOC 
Connecting to --port=/dev/cu.usbmodem14601 --baud=115200 
Ready.


### Import Libraries and setup ids

In [193]:
import board
import busio
from digitalio import DigitalInOut
from gc import collect,mem_free

In [194]:
# Unique feather identifier
FEATHER_ID = 1

In [195]:
print(mem_free())

21616


### TTN Connection

In [196]:
from adafruit_tinylora.adafruit_tinylora import TTN, TinyLoRa

In [197]:
# TinyLoRa/RFM9x Setup
spi = busio.SPI(board.SCK, MOSI=board.MOSI, MISO=board.MISO)
# pylint: disable=c-extension-no-member
cs = DigitalInOut(board.RFM9X_CS)
irq = DigitalInOut(board.RFM9X_D0)

i2c_bus = busio.I2C(board.SCL, board.SDA)

In [198]:
import config

devaddr = config.devaddr
nwkey = config.nwkey
appkey = config.appkey
#print(devaddr, nwkey, appkey)
 
ttn_config = TTN(devaddr, nwkey, appkey, country='AU')
 
lora = TinyLoRa(spi, cs, irq, ttn_config, channel=0)

In [199]:
collect()
print(mem_free())

9312


### Send Data

In [201]:
import analogio
vbat_voltage = analogio.AnalogIn(board.D9)

In [200]:
import adafruit_ccs811
ccs811 = adafruit_ccs811.CCS811(i2c_bus, address=0x5B)

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "adafruit_ccs811.py", line 76, in <module>
  File "adafruit_ccs811.py", line 96, in CCS811
MemoryError: 


In [202]:
def read_bat():
    return (vbat_voltage.value * 3.3) / 65536 * 2

In [203]:
def read_sensor():
    # Wait for the sensor to be ready and calibrate the thermistor
    while not ccs811.data_ready:
        pass
    temp = ccs811.temperature
    ccs811.temp_offset = temp - 25.0
    return ccs811.eco2, ccs811.tvoc, ccs811.temperature

In [150]:
def send_data():
    bme_d = bytearray(9)

    # Grab sensor data
    temp_val = 30
    humid_val = 70
    bat_val = int(read_bat() * 100)    

    bme_d[0] = FEATHER_ID
    # Temperature data
    bme_d[1] = (temp_val >> 8) & 0xff
    bme_d[2] = temp_val & 0xff
    # Humidity data
    bme_d[3] = (humid_val >> 8) & 0xff
    bme_d[4] = humid_val & 0xff
    # Battery data
    bme_d[5] = (bat_val >> 8) & 0xff
    bme_d[6] = bat_val & 0xff

    print('Sending packet...')
    lora.send_data(bme_d, len(bme_d), lora.frame_counter)
    print('Packet sent!')
    lora.frame_counter += 1

In [136]:
send_data()
#value = read_bat()
#print(value)
#for i in range(0,10):
#    send_data()
    # time.sleep(1 * 60)

Sending packet...
Packet sent!
